In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경변수 로드

True

In [32]:
from typing import List, Optional
from pydantic import BaseModel

# 👇 아래는 GraphState를 구성하는 모든 서브 모델들입니다.
class CandidateDocument(BaseModel):
    name: str
    summary: str
    domain: str

class EvaluationScores(BaseModel):
    team_trustworthiness: int
    customer_feedback: int
    risk_analysis: int
    founder_commitment: int

class MarketScores(BaseModel):
    market_size: int
    problem_fit: int
    willingness_to_pay: int
    revenue_model_clarity: int
    upside_potential: int

class MarketAnalysis(BaseModel):
    competitor_documents: Optional[List["CompetitorDocument"]] = []
    competitive_score: Optional[float] = None
    competitive_reasoning: Optional[str] = None

class CompetitorDocument(BaseModel):
    title: str
    url: str
    content: str

class InvestmentDecision(BaseModel):
    judgement: Optional[str] = None
    reasoning: Optional[str] = None

class GraphState(BaseModel):
    user_query: str
    domain: Optional[str] = None
    candidates_documents: Optional[List[CandidateDocument]] = []
    evaluation_scores: Optional[EvaluationScores] = None
    market_scores: Optional[MarketScores] = None  # ✅ 이 줄이 반드시 있어야 합니다
    evaluation_summary: Optional[str] = None
    market_analysis: Optional[MarketAnalysis] = None
    investment_decision: Optional[InvestmentDecision] = None
    final_report: Optional[str] = None

In [33]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)



In [42]:
def generate_markdown_from_state(state: GraphState) -> str:
    candidate = state.candidates_documents[0]
    scores = getattr(state, "market_scores", None)

    # 경쟁사 문서 중 1건만
    competitor_doc = (
        state.market_analysis.competitor_documents[0]
        if state.market_analysis and state.market_analysis.competitor_documents
        else None
    )

    return f"""# AI 스타트업 투자 평가 보고서

## 📌 스타트업 개요
- 이름: {candidate.name}
- 분야: {candidate.domain}
- 요약: {candidate.summary}

## 🧠 팀 기반 평가 점수 (0~10점)
- 창업자 및 팀 신뢰도: {state.evaluation_scores.team_trustworthiness}
- 고객 피드백: {state.evaluation_scores.customer_feedback}
- 기술/운영/법률 리스크: {state.evaluation_scores.risk_analysis}
- 창업자의 헌신도: {state.evaluation_scores.founder_commitment}

## 📊 시장성 평가 점수 (0~10점)
- 시장 크기: {getattr(scores, 'market_size', '정보 없음')}
- 문제 적합성: {getattr(scores, 'problem_fit', '정보 없음')}
- 지불 의사: {getattr(scores, 'willingness_to_pay', '정보 없음')}
- 수익 모델 명확성: {getattr(scores, 'revenue_model_clarity', '정보 없음')}
- 성공 시 기회 크기: {getattr(scores, 'upside_potential', '정보 없음')}

## 🥊 경쟁사 분석
- 경쟁력 점수: {state.market_analysis.competitive_score if state.market_analysis else "정보 없음"}
- 경쟁력 요약: {state.market_analysis.competitive_reasoning if state.market_analysis else "정보 없음"}
- 참고 문서: {competitor_doc.title if competitor_doc else "없음"}  
  ({competitor_doc.url if competitor_doc else "링크 없음"})

## 📝 요약 평가
{state.evaluation_summary}

## ✅ 최종 투자 판단
- 판단 결과: {state.investment_decision.judgement}
- 판단 근거: {state.investment_decision.reasoning}
"""

In [43]:
mock_graph_state = GraphState(
    user_query="AI 헬스케어 스타트업 알려줘",
    domain="헬스케어",
    candidates_documents=[
        CandidateDocument(
            name="Medibot",
            summary="Medibot은 AI 기반 진단 시스템으로 빠르고 정확한 진료 보조를 제공. AI 알고리즘을 통해 의료 데이터 분석을 지원.",
            domain="헬스케어"
        )
    ],
    evaluation_scores=EvaluationScores(
        team_trustworthiness=8,
        customer_feedback=7,
        risk_analysis=6,
        founder_commitment=9
    ),
    evaluation_summary=(
        "Medibot은 헬스케어 분야에 특화된 AI 기술 기반의 스타트업으로, 창업팀은 해당 산업에 대한 높은 전문성과 신뢰를 갖추고 있으며 (8점), "
        "초기 고객 피드백도 긍정적인 편입니다 (7점). 다만 기술 및 법률적 리스크에서 다소 우려되는 부분이 존재하며 (6점), "
        "창업자의 장기적인 헌신 의지는 높게 평가됩니다 (9점). "
        "종합적으로 Medibot은 유망한 AI 헬스케어 스타트업이며, 중장기적으로 투자 매력이 있는 기업으로 판단됩니다."
    ),
    market_analysis=MarketAnalysis(
        competitor_documents=[
            {
                "title": "헬스테크",
                "url": "https://fastercapital.com/ko/content/헬스테크-스타트업-전략.html",
                "content": "경쟁 및 차별화: 의료 기술 스타트업은 유사하거나 대안적인 제품 및 서비스를 제공하는 기존 기업, 동료 기업, 신규 기업 등 시장의 다른 플레이어와 경쟁하고 있습니다."
            },
            {
                "title": "[PDF]",
                "url": "https://www.techinvest.kr/board/bbs/download.php?bo_table=board2&wr_id=135&no=0",
                "content": "개발 및 생산비용이 스타트업 기업에게는 진입장벽으로 작용함."
            }
        ],
        competitive_score=3.5,
        competitive_reasoning=(
            "경쟁사와의 차별성이 뚜렷하지 않은 것으로 보입니다. 경쟁사들도 유사한 제품과 서비스를 제공하고 있으며, "
            "시장에 다른 플레이어들이 많이 존재합니다. 따라서 해당 스타트업의 차별성은 낮다고 평가됩니다."
        )
    ),
    investment_decision=InvestmentDecision(
        judgement="통과",
        reasoning=(
            "시장성과 기술력은 뛰어나지만, 경쟁사 대비 명확한 차별성은 부족합니다. "
            "다만 창업자의 헌신도와 고객 수요가 높고, 시장 규모가 크므로 중장기적 성장 가능성이 높아 투자 추천이 가능합니다."
        )
    ),
    final_report=None  # 나중에 Markdown 생성 함수로 채움
)

In [44]:
pip install pdfkit markdown2

Note: you may need to restart the kernel to use updated packages.


#### Markdown to PDF

In [48]:
markdown_text = generate_markdown_from_state(mock_graph_state)

In [52]:
pip install pdfkit markdown2

Note: you may need to restart the kernel to use updated packages.


In [54]:
import markdown2

def save_report_to_pdf(markdown_text: str, filename: str = "startup_report.pdf"):
    html_content = markdown2.markdown(markdown_text)

    styled_html = f"""
    <html>
    <head>
        <meta charset="utf-8">
        <style>
            body {{
                font-family: 'Nanum Gothic', sans-serif;
                line-height: 1.6;
                margin: 2em;
                color: #222;
            }}
            h1 {{
                font-size: 22pt;
                font-weight: bold;
                margin-top: 30px;
                border-bottom: 2px solid #ddd;
                padding-bottom: 5px;
            }}
            h2 {{
                font-size: 18pt;
                font-weight: bold;
                margin-top: 20px;
                color: #333;
            }}
            ul {{
                margin-left: 1.5em;
                padding-left: 0.5em;
            }}
            li {{
                margin-bottom: 6px;
                font-size: 12pt;
            }}
            p {{
                margin: 6px 0;
                font-size: 12pt;
            }}
            strong {{
                font-weight: bold;
                color: #000;
            }}
        </style>
    </head>
    <body>
    {html_content}
    </body>
    </html>
    """

    HTML(string=styled_html).write_pdf(filename)
    print(f"📄 PDF 저장 완료: {filename}")

In [55]:
# 1. 마크다운 생성
markdown_text = generate_markdown_from_state(mock_graph_state)

# 2. PDF 저장
save_report_to_pdf(markdown_text, "medibot_report.pdf")

NameError: name 'HTML' is not defined